In [1]:
import matplotlib.pyplot as plt
import numpy as np
from time import time
import cProfile
import io
import pstats
import plotly
import plotly.express as px

from src import helperfuncs
# from src import classify
from src_conv import classify_conv as classify
from src import cluster
from src_conv import forwardPass


folderPath = 'C:/My Documents/TUD-MCL/Semester 4/Thesis/Implementation/Data/Dataset-2/' # Maxime/' #sample 2/'
# folderPath = 'C:/My Documents/TUD-MCL/Semester 4/Thesis/Implementation/Data/Dataset-1/' 
# imgName = '18_04_27_Thomas_28618_0017.dm3'
imgName = 'Stack_zeolite4NaAF__111_001_1-10.tif'
rerun = 5
radius = 23

start = time()

startPosList= [[25,150],[75,250]]

load_start = time()
imgs = helperfuncs.loadData(folderPath=folderPath, fileName=imgName)
load_end = time()
print(f'Time for loading: {load_end - load_start} seconds!')

NumMainclasses=4
MinNumberInClass=4
# MaxNumberInClass=100
MaxNumberInClass=100*int(np.ceil(np.sqrt(len(imgs))))

Time for loading: 0.12492179870605469 seconds!


In [2]:
gen_start = time()
templates = helperfuncs.generateTemplates(startPosList=startPosList, imgs=imgs, radius=radius)
templates = helperfuncs.findDissimilarTemplates(templates = templates, imgs = imgs, radius = radius, minTemplateClasses = NumMainclasses)
gen_end = time()
print(f'Time for generating basic templates: {gen_end - gen_start} seconds!')


Time for generating basic templates: 1.1792969703674316 seconds!


In [3]:
rerun_ = rerun
classsify_start = time()
while rerun>0:
    templates = classify.tempfuncname(radius=radius, imgs=imgs, templates=templates, maxNumberInClass=MaxNumberInClass, minNumberInClass=MinNumberInClass)
    rerun-=1
classify_end = time()

1.9.1+cpu


In [ ]:
backplot_start = time()
backplot, min, max, templateMatchingResults = classify.backplotImg(radius, imgs, templates)
backplot_end = time()
print(f'Time for backplotting-1 : {backplot_end - backplot_start} seconds!')



for i in range(len(imgs)):
    plt.figure(figsize=(2*15, 2*7)) 
    ax1=plt.subplot(1,2,1)                    
    ax1.imshow(backplot[i][radius:-radius,radius:-radius],cmap=plt.cm.gray,vmin=min[i],vmax=max[i])
    ax1.set_title('backplot')
    ax1.axis('off')
    ax2=plt.subplot(1,2,2)                    
    ax2.imshow(imgs[i][radius:-radius,radius:-radius],cmap=plt.cm.gray,vmin=min[i],vmax=max[i])
    ax2.set_title('original image')
    ax2.axis('off')
    #plt.figure(figsize=(15, 12))  
    #plt.imshow(overlayclass[Mode][myindex],cmap=plt.cm.gist_rainbow)
    #plt.colorbar()
    # plt.show()

sort_start = time()
picDic = cluster.sortTemplates(imgs, templateMatchingResults, radius, templates)
sort_end = time()
print(f'Time for sort : {sort_end - sort_start} seconds!')

templateClassesMap = np.zeros((imgs[0].shape[0], imgs[0].shape[1]))
i=1
for pic in picDic:
    for p in pic:
        templateClassesMap[p["xIndex"]:p["xIndex"]+10,p["yIndex"]:p["yIndex"]+10]=i
    i+=1
fig = px.imshow(templateClassesMap, color_continuous_scale=px.colors.qualitative.Alphabet)
plotly.offline.plot(fig, filename='./charts/'+imgName+'-convtemplateClasses.html')


cluster_start = time()
centroidDic = cluster.cluster(radius, templates, picDic)
cluster_end = time()
print(f'Time for clustering : {cluster_end - cluster_start} seconds!')

backplot_start = time()
backplotFinal, min, max, overlayVariance = cluster.backplotFinal(centroidDic, picDic, imgs, radius, templateMatchingResults)    
backplot_end = time()
print(f'Time for backplotting-2 : {backplot_end - backplot_start} seconds!')

for i in range(len(imgs)):
    plt.figure(figsize=(2*15, 2*7)) 
    ax1=plt.subplot(1,2,1)                    
    ax1.imshow(backplotFinal[i][radius:-radius,radius:-radius],cmap=plt.cm.gray,vmin=min[i],vmax=max[i])
    ax1.set_title('backplot')
    ax1.axis('off')
    ax2=plt.subplot(1,2,2)                    
    ax2.imshow(imgs[i][radius:-radius,radius:-radius],cmap=plt.cm.gray,vmin=min[i],vmax=max[i])
    ax2.set_title('original image')
    ax2.axis('off')
    #plt.figure(figsize=(15, 12))  
    #plt.imshow(overlayclass[Mode][myindex],cmap=plt.cm.gist_rainbow)
    #plt.colorbar()
    # plt.show()

plt.savefig('C:/My Documents/TUD-MCL/Semester 4/Thesis/repo/img-denoiser/results/convTemp'+imgName+'-denoised.png')    

end = time()
print(f'Total time: {end - start} seconds!')

plt.figure(figsize=(20,20))



img = np.log(np.abs(np.fft.fftshift(np.fft.fft2(imgs[0][radius:-radius,radius:-radius]))))
ax1=plt.subplot(1,2,1)
ax1.imshow(img,cmap='gray')
ax1.axis('off')
ax1.set_title('FFT of original image')
img = np.log(np.abs(np.fft.fftshift(np.fft.fft2(backplotFinal[0][radius:-radius,radius:-radius]))))
ax1=plt.subplot(1,2,2)
ax1.imshow(img,cmap='gray')
ax1.axis('off')
ax1.set_title('FFT of denoised image in')
plt.show()
